# LLM Fine-tuning with Amazon Bedrock 
This notebook demonstrates the process of fine-tuning an LLM using Amazon Bedrock.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from typing import Optional
from langchain.schema import HumanMessage, SystemMessage
from langchain_aws import ChatBedrock

sys.path.append(os.pardir)
from core import BedrockModelFinetuner, get_s3_uri

Define file paths and AWS configurations.

In [ ]:
REGION_NAME = "us-west-2"
S3_BUCKET_NAME = "<YOUR-S3-BUCKET-NAME>"
DEFALUT_DATASETS_PREFIX = "datasets"

TRAIN_DATASET_PATH = "../assets/train_dataset.jsonl"
VALIDATION_DATASET_PATH = "../assets/validation_dataset.jsonl"


def get_dataset_uri(
    bucket_name: str, dataset_path: str, prefix: Optional[str] = DEFALUT_DATASETS_PREFIX
) -> str:
    filename = os.path.basename(dataset_path)
    key = filename if prefix is None else f"{prefix}/{filename}"
    return get_s3_uri(bucket_name, key)

### Step 1: Initialize Bedrock fine-tuner and prepare dataset URIs

In [ ]:
bedrock_model_finetuner = BedrockModelFinetuner(aws_region_name=REGION_NAME)

train_dataset_uri = get_dataset_uri(S3_BUCKET_NAME, TRAIN_DATASET_PATH)
validation_dataset_uri = get_dataset_uri(S3_BUCKET_NAME, VALIDATION_DATASET_PATH)

### Step 2: Fine-tune and deploy the model

In [ ]:
_ = bedrock_model_finetuner.finetune(train_dataset_uri, validation_dataset_uri)

In [ ]:
_ = bedrock_model_finetuner.deploy()

### Step 3: Test the fine-tuned model with a sample question

In [ ]:
chat = ChatBedrock(
    model_id=bedrock_model_finetuner.provisioned_model_arn,
    model_kwargs={
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 4096,
        "temperature": 0.0,
        "top_p": 0.9,
    },
    provider=bedrock_model_finetuner.provider,
    region_name=REGION_NAME,
)

In [ ]:
messages = [
    SystemMessage(
        content="""You're an AI analyzing NIKE's 2023 Form 10-K. Answer questions using only this document. 
        Be accurate, concise, and cite sources. State if info isn't available."""
    ),
    HumanMessage(
        content="""Based on the information in NIKE's 2023 Form 10-K, what are the main international markets 
        for NIKE products, and what challenges does the company face in these markets?"""
    ),
]
response = chat(messages)

print("Model response:", response.content + "\n")

### Step 4: Clean up resources

In [ ]:
_ = bedrock_model_finetuner.delete()